In [2]:
from transformers import BigBirdTokenizer, BigBirdForMaskedLM
import torch

# 모델과 토크나이저 로드
model_name = "google/bigbird-roberta-base"
tokenizer = BigBirdTokenizer.from_pretrained(model_name)
model = BigBirdForMaskedLM.from_pretrained(model_name)
print(model)
# 입력 텍스트
text = "Hugging Face의 Transformers 라이브러리를 사용하여 Big Bird 모델을 실행합니다."

# 텍스트를 토큰화하고 텐서로 변환
inputs = tokenizer(text, return_tensors="pt")

# 모델을 사용하여 예측
with torch.no_grad():
    outputs = model(**inputs)

# 결과 출력
logits = outputs.logits
print(logits)

BigBirdForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Attention type 'block_sparse' is not possible if sequence_length: 18 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size

BigBirdForMaskedLM(
  (bert): BigBirdModel(
    (embeddings): BigBirdEmbeddings(
      (word_embeddings): Embedding(50358, 768, padding_idx=0)
      (position_embeddings): Embedding(4096, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BigBirdEncoder(
      (layer): ModuleList(
        (0-11): 12 x BigBirdLayer(
          (attention): BigBirdAttention(
            (self): BigBirdBlockSparseAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): BigBirdSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
           

In [27]:
# BigBird (Inference) dataset prepare

inputs = [
    'I like reading [MASK].',
    'I like driving a [MASK].', 
    'The world is facing with [MASK] [MASK] crisis. We are all suffering from infectious diseases.',
]

answer = [
    "I like reading books.",
    "I like driving a car.",
    "The world is facing with pandemic crisis. We are all suffering from infectious diseases."
]

enconded_input = []
encoded_label = []
for _input, _answer in zip(inputs, answer):
    enconded_input.append(tokenizer(_input, return_tensors='pt'))
    encoded_label.append(tokenizer(_answer, return_tensors='pt')['input_ids'])


# Birbird (Inference) encoded_inputs과 encoded_labels를 사용하여 예측 수행하세요.
# 그리고 세 쌍의 손실, 예측결과 대조용 정답문장(을 출력하세요^^ ) 
with torch.no_grad():
    model.eval()
    for input, label in zip(enconded_input, encoded_label):
        outputs = model(**input, labels=label)
        # 로짓 추출
        logits = outputs.logits
        # 손실 추출
        loss = outputs.loss
        
        # 손실 출력. 이때 item() 은 텐서 loss 값을 파이썬 숫자로 추출
        print(f"loss: {loss.item()}")

        # len(logits[0])은 로짓의 첫 번째 차원의 개수이며
        # range(1, leg(logits[0]))은 1부터 로짓의 첫 번째 차원의 개수까지 범위

        # logits[0][1]는 로짓의 첫 번째 차원의 i번째 요소
        # argmax(-1)은 Logits[0][1]의 요소의 마지막 축(axis)에서 최대값을 갖는
        # 인덱스 추출
        # 해당 인덱스 기준으로 디코딩 후 그 결과를 순서대로 묶어서 출력
        print(f"prediction: {' '.join([tokenizer.decode(logits[0][i].argmax(-1)) for i in range(1, len(logits[0]))])}")
        print(f"answer: {tokenizer.decode(label[0][1:-1])}")
        print('\n')

loss: 10.482993125915527
prediction: i like reading it . i
answer: I like reading books.


loss: 8.306910514831543
prediction: its like driving a car . a
answer: I like driving a car.


loss: 4.74458646774292
prediction: the world is facing with a health crisis . we are all suffering from infectious diseases . .
answer: The world is facing with pandemic crisis. We are all suffering from infectious diseases.


